## Newton methods

#### Problem:
$$
f(\vec{x}) \rightarrow min,\\
f: \Omega \rightarrow \mathbb{R}, \\
\Omega \subset \mathbb{R^n}, f(\vec{x}) \mbox{ is convex}, \\
f(\vec{x}) \mbox{ - is twice diffirentiable on } \Omega\\
\vec{x_*} \in \Omega, f_{min} = f(\vec{x_*})
$$

We can greater efficency of finding $x_*$ if we use information not only about function gradient, but also Hessian $H(\vec{x})$

In simple variant on every k iteration function is approximated in neighborhood of point $\vec{x}_{k-1}$ by quadratic function $\phi_{k}(x)$ then $\vec{x}_k$ is found and the procedure continiues

By using Tailor series we can represent our function in neighborhood of point $x_{k}$ as
$$
f(\vec{x}) = f(\vec{x}_{k} + (\nabla f(\vec{x}_k), \vec{x} - \vec{x}_k) + \frac{1}{2}(H(\vec{x}_k)(\vec{x} - \vec{x}_k), \vec{x} - \vec{x}_k) + o(|\vec{x} - \vec{x}_k|)
$$

So our quadratic approximation $\phi_k(\vec{x})$ would be
$$
\phi_{k+1}(\vec{x}) = f(\vec{x}_{k} + (\nabla f(\vec{x}_k), \vec{x} - \vec{x}_k) + \frac{1}{2}(H(\vec{x}_k)(\vec{x} - \vec{x}_k), \vec{x} - \vec{x}_k)
$$

If our $H(\vec{x}_{k})$ is positive determined (function is convex), then $\vec{x}_{k+1}$ is single minimum of quadratic approximation and can be found using: 
$$ 
\nabla \phi_{k+1}(\vec{x}) = \nabla f(\vec{x}_k) + H(\vec{x}_k)(\vec{x} - \vec{x}_k) = \vec{0}
$$

Then we get
$$
\vec{x}_{k+1} = \vec{x}_{k}  - H^{-1}(\vec{x}_{k}) \nabla f(\vec{x}_{k})
$$

If our dimension number $n$ of space $\mathbb{R}$ is big enough, then finding $H^{-1}$ is very big problem. In this case it expedient to find minimum of $\phi_k(\vec{x})$ by using **gradient methods** or **conjugate directions method**
$\widetilde{\vec{x}}_k = argmin\{\phi_{k}(\vec{x})\}$ is just an approximation, using this we can build *relaxational sequence* 

$$
\vec{x}_{k} = \vec{x}_{k-1} + \lambda_k(\widetilde{\vec{x}}_{k} - \vec{x}_{k-1}) = \vec{x}_{k-1} + \lambda_k\vec{p}_{k} \\ 
\vec{p}_k = -H^{-1}(\vec{x}_{k-1}) \nabla f(\vec{x}_{k-1}) \mbox{ - direction of descent}
$$

We can find $\lambda_k$ different ways, for example find $argmin\{f(\vec{x}_{k-1} + \lambda_k\vec{p}_k\}$ or by method of step splitting


In [7]:
import matplotlib as mplib
import math as m
import numpy as np
from numpy.linalg import norm
from scipy import linalg
from scipy import sparse
from scipy import optimize
import matplotlib.pyplot as plt
from scipy.misc import derivative

from onedim_optimize import quadratic_approx, fibbonaci_method, middle_point_method, upgraded_newton, qubic_approx, brent_optimize
from scipy.optimize import approx_fprime, line_search
import matplotlib.animation as pltanimation
from test_functions import *
from animations import Animate3D

def toOneParamFunc(f, x, w):
    return lambda p: f(x + p*w) 

def argmin(f, a, b, eps, onedim_opti):
#     fig, ax = plt.subplots()
#     ax.plot(np.linspace(a, b, 1000), [f(y) for y in np.linspace(a, b, 1000)])
    f_ev, j_ev = 0, 0
#     x, f_ev =  upgraded_newton(f, a, b, eps)
    x, f_ev = onedim_opti(f, a, b, eps)
#     ax.scatter(x, f(x))
    return x, f_ev

def approx_gradient(f, eps):
    return lambda x: approx_fprime(x, f, eps)

def hessian_in_point(x, f, grad, eps):
     gr = grad(x)
     n = len(gr) 
     hes = []
     for i in range (0, n):
        x_delta = np.array(x[:])
        x_delta[i] = x_delta[i] + eps
        gr_delta = grad(x_delta)
        partials = np.array([partial_der(gr_delta[j], part, eps, j) for j,part in enumerate(gr)])
        hes.append(partials)
     return np.array(hes)

def hessian(f, grad, eps):
    return lambda x: hessian_in_point(x, f, grad, eps) 

def optimization_result(title, fmin, xmin, K, f_ev, j_ev, h_ev = None, res=None):
    print(f"""
{title}
Optimization {res}
x minimum: {xmin},
f minimum: {fmin},
number of iterations: {K},
number of function evaluations: {f_ev},
number of gradient evaluations: {j_ev},
{f"number of hessian evaluations: {h_ev}" if h_ev != None else ''}
""") if res == 'succes' else print(f"""{title}\nOptimization {res}""")

### Test functions

In [2]:

test_sqrt1 = [
    danilov,
    danilov_gradient,
    np.array([-2, 2]),
    0.001,
    'Square root func 1 test. Starting point (-2, 2)' 
]

test_sqrt2 = [
    danilov,
    danilov_gradient,
    np.array([4, 3]),
    0.001,
    'Square root func 1 test. Starting point (4, 3)' 
]

test_rosen1 = [
    rosenbrok,
    rosen_gradient,
    np.array([-2, -1]),
    1e-4,
    'Rosenbrock1 test. Starting point (-2, -1)'
]

test_rosen2 = [
    rosenbrok,
    rosen_gradient,
    np.array([-3, 4]),
    1e-4,
    'Rosenbrock2 test. Starting point (-3, 4)'
]

test_rosen3 = [
    rosenbrok,
    rosen_gradient,
    np.array([3, 3]),
    1e-4,
    'Rosenbrock3 test. Starting point (3, 3)'
]

test_himmel1 = [
    himmelblau,
    himmel_gradient,
    np.array([0, -4]),
    1e-4,
    'Himmelblau1 test. Starting point (0, -4)'
]

test_himmel2 = [
    himmelblau,
    himmel_gradient,
    np.array([10, 21]),
    1e-4,
    'Himmelblau1 test. Starting point (10, 21)'
]

test_himmel3 = [
    himmelblau,
    himmel_gradient,
    np.array([-5, 17]),
    1e-4,
    'Himmelblau1 test. Starting point (-5, 17)'
]

# test_rastrigin = [
#     rastrigin,
#     approx_gradient(rastrigin, np.float64(1e-8)),
#     np.array([2, 1]),
#     1e-4
# ]

# test_ackley = [
#     ackley,
#     approx_gradient(ackley, np.float64(1e-9)),
#     np.array([1, 1]),
#     1e-4
# ]

# test_sphere = [
#     sphere,
#     approx_gradient(sphere, np.float64(1e-9)),
#     np.array([-3, 3]),
#     1e-5,
#     [[-3, 3], [0, 10]]
# ]

# test_beale = [
#     beale,
#     approx_gradient(beale, np.float64(1e-9)),
#     np.array([3, 1.5]),
#     1e-3,
#     [[-0.01, 800], [2.9, 1.6]]
# ]

# test_goldstein = [
#     goldstein_price,np.array([2, 1]),
#     approx_gradient(goldstein_price, np.float64(1e-9)),
#     np.array([-1.3, 1]),
#     1e-5,
#     [[-1.5, 1], [0, 50000]]
# ]

# test_booth = [
#     booth,
#     approx_gradient(booth, np.float64(1e-8)),
#     np.array([5, 3]),
#     1e-5,
#     [[0, 8], [0, 700]]
# ]

# test_bukin = [
#     bukin,
#     approx_gradient(bukin, np.float64(1e-8)),
#     np.array([-10.5, 1.5]),
#     1e-5
# ]

# test_himmel = [
#     himmelblau,
#     approx_gradient(himmelblau, np.float64(1e-8)),
#     np.array([0, -4]),
#     1e-5,
#     [[-4, 4], [-0.1, 280]]
# ]

# test_egg = [
#     eggholder,
#     approx_gradient(eggholder, np.float64(1e-8)),
#     np.array([353, -200]),
#     1e-7
# ]

# test_cross = [
#     cross,
#     approx_gradient(cross, np.float64(1e-8)),
#     np.array([2, -2]),
#     1e-4
# ]

### Newton method

The common Newton method is to find our $H^{-1}$ matrix and than build relaxetion sequence by this rule:
$$
\vec{x}_{k+1} = \vec{x}_{k}  - H^{-1}(\vec{x}_{k}) \nabla f(\vec{x}_{k})
$$

But there is a problem with matrix $H$, it needs to be always positive determinated or $H^{-1}$ won't exist.

To solve this problem, let's check if $H$ is positive determinated, if not, then let's pick $\eta$_k, such that:
$$
\widetilde{H}_k = \eta_kI_n + H(\vec{x}_{k-1})
$$
$\widetilde{H}$ is positive determinated matrix, that we pick instead of $H$


In [3]:
def common_newton(f, gr, x, epsilon, title, hess):
    try:
        anim = Animate3D(f, x, title)
        w = -gr(x)
        k = 0
        f_ev, j_ev, h_ev = 0, 0, 0
        while(norm(w) > epsilon):
            H = hess(x)
            j_ev += 1
            h_ev += 1
    #         print(x, w)
            h = linalg.solve(H, w)
            x = x + h
            anim.add(x)
            w = -gr(x)
            k += 1
            if k == 100:
                return f(x), x, k, f_ev, j_ev, h_ev, anim, "fail"
        return f(x), x, k, f_ev, j_ev, h_ev, anim, "succes"
    except: 
        return f(x), x, k, f_ev, j_ev, h_ev, anim, "fail"


In [4]:
fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = common_newton(*test_sqrt1, hess=danilov_hessian)
optimization_result(test_sqrt1[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=3000).save('examples/Sqrt/Sqrt1-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = common_newton(*test_sqrt2, hess=danilov_hessian)
optimization_result(test_sqrt2[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=3000).save('examples/Sqrt/Sqrt2-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = common_newton(*test_rosen1, hess=rosen_hessian)
optimization_result(test_rosen1[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=6000).save('examples/Rosenbrock/Rosenbrock1-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = common_newton(*test_rosen2, hess=rosen_hessian)
optimization_result(test_rosen2[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=6000).save('examples/Rosenbrock/Rosenbrock2-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = common_newton(*test_rosen3, hess=rosen_hessian)
optimization_result(test_rosen3[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=6000).save('examples/Rosenbrock/Rosenbrock3-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = common_newton(*test_himmel1, hess=himmel_hessian)
optimization_result(test_himmel1[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel1-Fastest-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = common_newton(*test_himmel2, hess=himmel_hessian)
optimization_result(test_himmel2[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel2-Fastest-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = common_newton(*test_himmel3, hess=himmel_hessian)
optimization_result(test_himmel3[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel3-Fastest-Newton.gif')

Square root func 1 test. Starting point (-2, 2)
Optimization fail
Square root func 1 test. Starting point (4, 3)
Optimization fail

Rosenbrock1 test. Starting point (-2, -1)
Optimization succes
x minimum: [1. 1.],
f minimum: 4.378375239202921e-27,
number of iterations: 5,
number of function evaluations: 0,
number of gradient evaluations: 5,
number of hessian evaluations: 5


Rosenbrock2 test. Starting point (-3, 4)
Optimization succes
x minimum: [1. 1.],
f minimum: 1.6568659570877452e-24,
number of iterations: 5,
number of function evaluations: 0,
number of gradient evaluations: 5,
number of hessian evaluations: 5


Rosenbrock3 test. Starting point (3, 3)
Optimization succes
x minimum: [1. 1.],
f minimum: 2.923222691909612e-28,
number of iterations: 5,
number of function evaluations: 0,
number of gradient evaluations: 5,
number of hessian evaluations: 5


Himmelblau1 test. Starting point (0, -4)
Optimization succes
x minimum: [-0.12796134 -1.953715  ],
f minimum: 178.33723920192747,
nu

/home/urban/source/repos/dzshki/optmehods/lab1/test_functions.py:29: RuntimeWarning: overflow encountered in power
  danilov_hessian = lambda x: np.array([[np.divide(4, np.sqrt(x[0]**2 + x[1]**2 + 1)) - np.divide(4*x[0]**2, np.power(x[0]**2 + x[1]**2 + 1, 1.5)), -np.divide(4*x[0]**2, np.power(x[0]**2 + x[1]**2 + 1, 1.5))],
/home/urban/source/repos/dzshki/optmehods/lab1/test_functions.py:30: RuntimeWarning: overflow encountered in power
  [-np.divide(4*x[0]*x[1], np.power(x[0]**2 + x[1]**2 + 1, 1.5)), np.divide(4, np.sqrt(x[0]**2 + x[1]**2 + 1)) - np.divide(4*x[1]**2, np.power(x[0]**2 + x[1]**2 + 1, 1.5))]])


In [5]:
def newton_upgraded(f, gr, x, epsilon, title, hess, onedim_opt):
    try:
        anim = Animate3D(f, x, title)
        w = -gr(x)
        phi = toOneParamFunc(f, x, w)
        k = 0
        f_ev, j_ev, h_ev = 0, 0, 0
    #     print(x)
        while(norm(w) > epsilon):
            H = hess(x)
            h_ev += 1
            j_ev += 1
            h = linalg.solve(H, w)
            phi = toOneParamFunc(f, x, h)
            l, i = argmin(phi, 0, 600, np.divide(epsilon, 1e5), onedim_opt) 
            f_ev += i
            x = x + l * h
    #         print(x)
            anim.add(x)
            w = -gr(x)
            k += 1
            if k == 100:
                return f(x), x, k, f_ev, j_ev, h_ev, anim, 'fail'
        return f(x), x, k, f_ev, j_ev, h_ev, anim, 'succes'
    except:
        return f(x), x, k, f_ev, j_ev, h_ev, anim, 'fail'

In [8]:
fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = newton_upgraded(*test_sqrt1, hess=danilov_hessian, onedim_opt=brent_optimize)
optimization_result(test_sqrt1[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=5000).save('examples/Sqrt/Sqrt1-Upgr-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = newton_upgraded(*test_sqrt2, hess=danilov_hessian, onedim_opt=brent_optimize)
optimization_result(test_sqrt2[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=5000).save('examples/Sqrt/Sqrt2-Upgr-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = newton_upgraded(*test_rosen1, hess=rosen_hessian, onedim_opt=brent_optimize)
optimization_result(test_rosen1[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=8000).save('examples/Rosenbrock/Rosenbrock1-Upgr-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = newton_upgraded(*test_rosen2, hess=rosen_hessian, onedim_opt=brent_optimize)
optimization_result(test_rosen2[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=8000).save('examples/Rosenbrock/Rosenbrock2-Upgr-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = newton_upgraded(*test_rosen3, hess=rosen_hessian, onedim_opt=brent_optimize)
optimization_result(test_rosen3[4], fmin, xmin, K, f_ev, j_ev, h_ev, res)
# a = anim.get_animation(duration=8000).save('examples/Rosenbrock/Rosenbrock3-Upgr-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = newton_upgraded(*test_himmel1, hess=himmel_hessian, onedim_opt=brent_optimize)
optimization_result(test_himmel1[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel1-Upgr-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = newton_upgraded(*test_himmel2, hess=himmel_hessian, onedim_opt=brent_optimize)
optimization_result(test_himmel2[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel2-Upgr-Newton.gif')

fmin, xmin, K, f_ev, j_ev, h_ev, anim, res = newton_upgraded(*test_himmel3, hess=himmel_hessian, onedim_opt=brent_optimize)
optimization_result(test_himmel3[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel3-Upgr-Newton.gif')


Square root func 1 test. Starting point (-2, 2)
Optimization succes
x minimum: [-0.30151656 -0.60299587],
f minimum: 3.3166247913499816,
number of iterations: 4,
number of function evaluations: 70,
number of gradient evaluations: 4,
number of hessian evaluations: 4


Square root func 1 test. Starting point (4, 3)
Optimization succes
x minimum: [-0.30146196 -0.60297794],
f minimum: 3.316624795720703,
number of iterations: 4,
number of function evaluations: 88,
number of gradient evaluations: 4,
number of hessian evaluations: 4


Rosenbrock1 test. Starting point (-2, -1)
Optimization succes
x minimum: [0.99999977 0.9999995 ],
f minimum: 1.9783972876656746e-13,
number of iterations: 14,
number of function evaluations: 206,
number of gradient evaluations: 14,
number of hessian evaluations: 14


Rosenbrock2 test. Starting point (-3, 4)
Optimization succes
x minimum: [0.9999996  0.99999923],
f minimum: 3.2421921095134094e-13,
number of iterations: 17,
number of function evaluations: 261,
nu

In [9]:
def count_next_matrix(A, dw, dx):
    y = dx
    z = A.dot(dw)
    first_part = np.dot(np.transpose([y]), [dx]) * np.divide(1, dw.dot(y))
    sec_part =  np.dot(np.transpose([z]), [dw]).dot(A) * np.divide(1, z.dot(dw))
    return A - first_part - sec_part

def count_next_matrix_2(A, dw,dx):
    I = np.identity(len(dx))
    r = np.divide(1, np.dot(dx, dw))
    C1 = (I - r*(np.dot(np.transpose([dw]), [dx])))
    C2 = (I - r*(np.dot(np.transpose([dx]), [dw])))
    first_part = C1.dot(A).dot(C2)
    sec_part =  r*np.dot(np.transpose([dx]), [dx])
    return first_part - sec_part

def DFP(f, grad, x, epsilon, title, onedim_opt):
    try:
        w2 = -grad(x)
        p = w2
        f_ev = 0
        j_ev = 1
        anim = Animate3D(f, x, title)
        phi = toOneParamFunc(f, x, w2)
        x1 = x
        k = 1
#         n = 0
        A = np.identity(len(x))
        l, i = argmin(phi, 0, 600, np.divide(epsilon, 1e4), onedim_opt)
        f_ev += i
    #     print(x1, f(x1), l, w2)
        x2 = x1 + l * p
        anim.add(x2)
        while(norm(w2) > epsilon):
            w1 = w2
            w2 = -grad(x2)
            A = count_next_matrix(A, w2 - w1, x2 - x1)
            p = np.dot(A, w2)
            x1 = x2
            phi = toOneParamFunc(f, x2, p)
            l, i = argmin(phi, 0, 600, np.divide(epsilon, 1e4), onedim_opt)
    #         print(x1, f(x1), l, w2)
            x2 = x1 + l * p
            anim.add(x2)
            f_ev += i
            j_ev += 1
            k += 1
    #         print("A", A, "\ninverted Hessian", np.linalg.inv(rosen_hessian(x2)))
        return f(x2), x2, k, f_ev, j_ev, anim, 'succes'
    except:
        return f(x2), x2, k, f_ev, j_ev, anim, 'fail'

        

In [11]:
fmin, xmin, K, f_ev, j_ev, anim, res = DFP(*test_sqrt1, onedim_opt=brent_optimize)
optimization_result(test_sqrt1[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=5000).save('examples/Sqrt/Sqrt1-DFP.gif')

fmin, xmin, K, f_ev, j_ev, anim, res = DFP(*test_sqrt2, onedim_opt=brent_optimize)
optimization_result(test_sqrt2[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=5000).save('examples/Sqrt/Sqrt2-DFP.gif')

fmin, xmin, K, f_ev, j_ev, anim, res = DFP(*test_rosen1, onedim_opt=brent_optimize)
optimization_result(test_rosen1[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Rosenbrock/Rosenbrock1-DFP.gif')

fmin, xmin, K, f_ev, j_ev, anim, res = DFP(*test_rosen2, onedim_opt=brent_optimize)
optimization_result(test_rosen2[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=10000).save('examples/Rosenbrock/Rosenbrock2-DFP.gif')

fmin, xmin, K, f_ev, j_ev, anim, res = DFP(*test_rosen3, onedim_opt=brent_optimize)
optimization_result(test_rosen3[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Rosenbrock/Rosenbrock3-DFP.gif')

fmin, xmin, K, f_ev, j_ev, anim, res = DFP(*test_himmel1, onedim_opt=brent_optimize)
optimization_result(test_himmel1[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel1-DFP.gif')

fmin, xmin, K, f_ev, j_ev, anim, res = DFP(*test_himmel2, onedim_opt=brent_optimize)
optimization_result(test_himmel1[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel2-DFP.gif')

fmin, xmin, K, f_ev, j_ev, anim, res = DFP(*test_himmel3, onedim_opt=brent_optimize)
optimization_result(test_himmel1[4], fmin, xmin, K, f_ev, j_ev, res=res)
# a = anim.get_animation(duration=8000).save('examples/Himmelblau/Himmel3-DFP.gif')


Square root func 1 test. Starting point (-2, 2)
Optimization succes
x minimum: [-0.30151057 -0.6030228 ],
f minimum: 3.3166247903563697,
number of iterations: 5,
number of function evaluations: 76,
number of gradient evaluations: 5,



Square root func 1 test. Starting point (4, 3)
Optimization succes
x minimum: [-0.30151134 -0.60302269],
f minimum: 3.3166247903553994,
number of iterations: 6,
number of function evaluations: 102,
number of gradient evaluations: 6,



Rosenbrock1 test. Starting point (-2, -1)
Optimization succes
x minimum: [1. 1.],
f minimum: 7.456949295253838e-26,
number of iterations: 17,
number of function evaluations: 317,
number of gradient evaluations: 17,



Rosenbrock2 test. Starting point (-3, 4)
Optimization succes
x minimum: [1. 1.],
f minimum: 9.970462284894945e-29,
number of iterations: 19,
number of function evaluations: 354,
number of gradient evaluations: 19,



Rosenbrock3 test. Starting point (3, 3)
Optimization succes
x minimum: [1. 1.],
f minimum: 2